In [1]:
import numpy 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from tensorflow.python.keras import backend

Using TensorFlow backend.


In [21]:
data = pd.read_csv("C:/Users/user/EWHA BIGDATA/FINAL_DATA.csv", encoding='euc-kr', parse_dates=['Date'])
seed = 5
numpy.random.seed(seed)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107215 entries, 0 to 107214
Data columns (total 9 columns):
Week        107215 non-null object
gender      107215 non-null object
age         107215 non-null object
temp        107215 non-null float64
rain        107215 non-null float64
wind        107215 non-null float64
dust        107215 non-null float64
temp_dif    107215 non-null float64
call_num    107215 non-null object
dtypes: float64(5), object(4)
memory usage: 7.4+ MB


In [36]:
data.tail()

,Week,gender,age,temp,rain,wind,dust,temp_dif,call_num
107210,5,2,2,3.7,0.0,1.4,56.0,8.6,0
107211,5,2,3,3.7,0.0,1.4,56.0,8.6,1
107212,5,2,4,3.7,0.0,1.4,56.0,8.6,1
107213,5,2,5,3.7,0.0,1.4,56.0,8.6,1
107214,5,2,6,3.7,0.0,1.4,56.0,8.6,1


In [23]:
data['call_num'] = data['call_num'].astype(object)

In [25]:
data.drop(['mean','call','GU','Date','temp_H','temp_L','Unnamed: 0'], axis='columns', inplace=True)

In [32]:
gender = {'남': 1,'여': 2}
data.gender = [gender[item] for item in data.gender] 

In [34]:
age = {'10대': 1,'20대': 2,'30대': 3,'40대': 4,'50대': 5,'60대이상': 6}
data.age = [age[item] for item in data.age] 

In [35]:
Week = {'월': 1,'화': 2,'수': 3,'목': 4,'금': 5,'토': 6,'일': 7}
data.Week = [Week[item] for item in data.Week] 

In [37]:
prediction_var = ['Week','temp','rain','wind','temp_dif','dust','gender','age']
X = data[prediction_var].values
Y = data.call_num.values

In [38]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [41]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(3, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model. We use the the logarithmic loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [42]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=10, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 80.03% (0.85%)
